# JEM092 Asset Pricing
# Seminar 12
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 11.5.2022

In [ ]:
# Import third-party packages.
library(lubridate)
library(quantmod)
library(sandwich)
library(xts)

# Data preparation

In [ ]:
load('seminar_11.RData')
ls()

In [ ]:
START_DATE <- '2005-01-01'
END_DATE <- '2021-03-31'

# Load stock specific data.
abs_return_to_nominal_ratios <- c()
for (i in 1:length(names(daily_returns))) {
    ticker <- as.character(names(daily_returns)[i])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_abs_return_to_nominal_ratios <- abs(daily_returns[, ticker]) / (stock_data[, 5] * stock_data[, 6] / 1000000)
    colnames(stock_abs_return_to_nominal_ratios) <- ticker
    abs_return_to_nominal_ratios <- cbind(abs_return_to_nominal_ratios, stock_abs_return_to_nominal_ratios)
}

In [ ]:
abs_return_to_nominal_ratios[1:5, 1:3]

## Compute illiquidity

In [ ]:
# Initialize an xts object for storing illiquidities.
monthly_illiquidities <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_illiquidities) <- names(monthly_returns)

# Iterate over rows and compute illiquidities.
for (i in 1:nrow(monthly_illiquidities)) {
    end_day <- index(monthly_illiquidities[i])
    start_day <- end_day %m-% months(12)
    dates <- paste0(start_day, '/', end_day)
    ratios <- abs_return_to_nominal_ratios[dates]

    # Compute illiquidities for each stock.
    for (j in 1:ncol(monthly_illiquidities)) {
        stock_ratios <- ratios[, j]

        # Save the given illiquidity to `monthly_illiquidities`.
        monthly_illiquidities[i, j] <- log(1 + mean(stock_ratios))
    }
}

In [ ]:
monthly_illiquidities[1:5, 1:3]

## Compute skewness

In [ ]:
# Initialize an xts object for storing skewnesses.
monthly_skewnesses <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_skewnesses) <- names(monthly_returns)

# Iterate over rows and compute skewnesses.
for (i in 1:nrow(monthly_skewnesses)) {
    end_day <- index(monthly_skewnesses[i])
    start_day <- end_day %m-% months(1)
    dates <- paste0(start_day, '/', end_day)
    returns <- daily_returns[dates]

    # Compute skewnesses for each stock.
    for (j in 1:ncol(monthly_skewnesses)) {
        stock_returns <- returns[, j]

        average_stock_return <- mean(stock_returns)
        nominator <- mean((stock_returns - average_stock_return) ^ 3)
        denominator <- (mean((stock_returns - average_stock_return) ^ 2)) ^ (3 / 2)

        # Save the given skewness to `monthly_skewnesses`.
        monthly_skewnesses[i, j] <- nominator / denominator
    }
}

In [ ]:
monthly_skewnesses[1:5, 1:3]

## Compute volatility

In [ ]:
# Initialize an xts object for storing volatilities.
monthly_volatilities <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_volatilities) <- names(monthly_returns)

# Iterate over rows and compute volatilities.
for (i in 1:nrow(monthly_volatilities)) {
    end_day <- index(monthly_volatilities[i])
    start_day <- end_day %m-% months(1)
    dates <- paste0(start_day, '/', end_day)
    returns <- daily_returns[dates]

    # Compute volatilities for each stock.
    for (j in 1:ncol(monthly_volatilities)) {
        stock_returns <- returns[, j]

        average_stock_return <- mean(stock_returns)
        nominator <- sum((stock_returns - average_stock_return) ^ 2)
        denominator <- length(stock_returns) - 1

        # Save the given volatility to `monthly_volatilities`.
        monthly_volatilities[i, j] <- 100 * sqrt(nominator / denominator) * sqrt(12)
    }
}

In [ ]:
monthly_volatilities[1:5, 1:3]

# Define functions for portfolio sorts and Fama-MacBeth regressions

## Univariate portfolio sorts

In [ ]:
perform_univariate_portfolio_sort <- function (sort_variable_data, returns, market_caps, n) {
    diff_portfolio <- paste(n, '- 1')

    # Initialize an xts object for storing portfolio returns.
    portfolio_returns <- as.xts(
        matrix(
            nrow = nrow(sort_variable_data) - 1,
            ncol = n + 1
        ),
        order.by = index(sort_variable_data[2:nrow(sort_variable_data)])
    )
    names(portfolio_returns) <- c(1:n, diff_portfolio)

    # Iterate over rows, find breakpoints and compute monthly returns within the given value breakpoints.
    for (i in 1:nrow(portfolio_returns)) {
        current_month <- index(portfolio_returns[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(returns)) {
            next
        }

        # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
        month_returns <- returns[next_month]
        month_bm_ratios <- sort_variable_data[current_month]
        month_market_caps <- market_caps[next_month]
        # Replace NA market caps with 0s so that such observations have no weights.
        month_market_caps[is.na(month_market_caps)] <- 0

        breakpoints <- quantile(month_bm_ratios, 0:n/n, na.rm = TRUE)
        not_na <- !is.na(month_bm_ratios)

        for (j in 1:n) {
            filter <- (breakpoints[[j]] < month_bm_ratios) & (month_bm_ratios < breakpoints[[j + 1]]) & not_na
            # Compute weighted average portfolio returns.
            portfolio_returns[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
        }
        portfolio_returns[i, diff_portfolio] <- portfolio_returns[i, n] - portfolio_returns[i, 1]
    }

    # Compute overall average returns within portfolios and their standard errors.
    results_bm_ratios <- as.data.frame(matrix(nrow = 2, ncol = n + 1))
    names(results_bm_ratios) <- c(1:n, diff_portfolio)

    for (i in 1:ncol(results_bm_ratios)) {
        model <- lm(na.omit(portfolio_returns[, i]) ~ 1)
        results_bm_ratios[1, i] <- model$coefficients[[1]]
        results_bm_ratios[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
    }

    results_bm_ratios
}

## Fama-MacBeth regressions

In [ ]:
perform_fama_macbeth_regression <- function (data_explained, data_explanatory) {
    # Initialize an xts object for storing regression results.
    fm_results <- as.xts(
        matrix(
            nrow = nrow(data_explained) - 1,
            ncol = 1 + length(data_explanatory)
        ),
        order.by = index(data_explained[2:nrow(data_explained)])
    )
    column_names <- c('Intercept')
    column_names <- c(column_names, names(data_explanatory))
    names(fm_results) <- column_names

    # Iterate over rows and perform cross-sectional regressions.
    for (i in 1:nrow(fm_results)) {
        current_month <- index(fm_results[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(data_explained)) {
            next
        }

        # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
        month_data_explained <- data_explained[next_month]
        month_data_explanatory <- c()
        for (j in 1:length(data_explanatory)) {
            if (!current_month %in% index(data_explanatory[[j]])) {
                move_to_next_month <- TRUE
                break
            }
            move_to_next_month <- ncol(data_explanatory[[j]][current_month]) == sum(is.na(data_explanatory[[j]][current_month]))
            if (move_to_next_month) {
                break
            }
            colnames(month_data_explanatory)
            cross_section <- t(data_explanatory[[j]][current_month])
            colnames(cross_section) <- names(data_explanatory)[j]

            cross_section[which(cross_section == -Inf)] = NA
            cross_section[which(cross_section == Inf)] = NA

            month_data_explanatory <- cbind(month_data_explanatory, cross_section)
        }

        if (move_to_next_month) {
            next
        }

        transposed_explained <- t(month_data_explained)
        colnames(transposed_explained) <- c('explained')
        equation <- paste('explained ~ 1 + ', paste0(colnames(month_data_explanatory), collapse = ' + '))
        all_data <- na.omit(as.data.frame(cbind(transposed_explained, month_data_explanatory)))

        # Save regression coefficients and other statistics.
        model <- lm(equation, data = all_data)
        for (k in 1:ncol(fm_results)) {
            fm_results[i, k] <- model$coefficients[[k]]
        }
    }

    # Compute time series means, standard errors.
    final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = ncol(fm_results)))
    names(final_fm_results) <- colnames(fm_results)

    for (i in 1:ncol(final_fm_results)) {
        model <- lm(na.omit(fm_results[, i]) ~ 1)
        final_fm_results[1, i] <- model$coefficients[[1]]
        final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
    }

    final_fm_results
}

# Liquidity

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
perform_univariate_portfolio_sort(monthly_illiquidities, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regression analysis

In [ ]:
data <- list(monthly_illiquidities)
names(data) <- c('Illiquidity')
perform_fama_macbeth_regression(monthly_returns, data)

## Fama-MacBeth regressions of returns on illiquidity, reversal, momentum, BM ratios, betas and sizes

In [ ]:
data <- list(monthly_illiquidities, monthly_reversal, monthly_momentum, monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Illiquidity', 'Reversal', 'Momentum', 'Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)

# Skewness

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
perform_univariate_portfolio_sort(monthly_skewnesses, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regression analysis

In [ ]:
data <- list(monthly_skewnesses)
names(data) <- c('Skewness')
perform_fama_macbeth_regression(monthly_returns, data)

## Fama-MacBeth regressions of returns on skewness, illiquidity, reversal, momentum, BM ratios, betas and sizes

In [ ]:
data <- list(monthly_skewnesses, monthly_illiquidities, monthly_reversal, monthly_momentum, monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Skewness', 'Illiquidity', 'Reversal', 'Momentum', 'Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)

# Volatility

## Portfolio analysis using monthly univariate decile portfolios

In [ ]:
perform_univariate_portfolio_sort(monthly_volatilities, monthly_returns, monthly_market_caps, 10)

## Fama-MacBeth regression analysis

In [ ]:
data <- list(monthly_volatilities)
names(data) <- c('Volatility')
perform_fama_macbeth_regression(monthly_returns, data)

## Fama-MacBeth regressions of returns on volatility, skewness, illiquidity, reversal, momentum, BM ratios, betas and sizes

In [ ]:
data <- list(monthly_volatilities, monthly_skewnesses, monthly_illiquidities, monthly_reversal, monthly_momentum, monthly_bm_ratios, monthly_betas_24m, monthly_sizes)
names(data) <- c('Volatility', 'Skewness', 'Illiquidity', 'Reversal', 'Momentum', 'Value', 'Beta', 'Size')
perform_fama_macbeth_regression(monthly_returns, data)